In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import snapatac2 as sn
from pathlib import Path
import numpy as np
plt.rcParams["figure.figsize"] = (30,30)
import scipy.stats as stats
import pickle
import time
import timeit
SEED = 0
np.random.seed(SEED)
from sklearn.decomposition import PCA

In [2]:
data_path = Path("/data/toulouse/scMultiSim/run_04")
out_path = Path("/data/toulouse/bicycle/notebooks/experiments/masking/data")
run_id = "scMultiSim_data/run_04"

In [3]:
atac_base = pd.read_csv(data_path/"atac_counts.csv", index_col=0).to_numpy(np.float32)
grn = pd.read_csv(data_path/"geff.csv", index_col=0).to_numpy(np.float32)
region_to_gene = pd.read_csv(data_path/"region_to_gene.csv", index_col=0).to_numpy(np.float32)
region_to_tf = pd.read_csv(data_path/"region_to_tf.csv", index_col=0).to_numpy(np.float32)
atacseq_obs = pd.read_csv(data_path/"atacseq_obs.csv", index_col=0).to_numpy(np.float32)


filt_grn = pd.read_csv(out_path/run_id/"filtered_grn.csv", index_col=0).to_numpy(np.float32)
filt_region_to_gene = pd.read_csv(out_path/run_id/"filtered_region_to_gene.csv", index_col=0).to_numpy(np.float32)
filt_region_to_tf = pd.read_csv(out_path/run_id/"filtered_region_to_tf.csv", index_col=0).to_numpy(np.float32)
filt_atacseq_obs = pd.read_csv(out_path/run_id/"processed_atac.csv", index_col=0).to_numpy(np.float32)

In [ ]:
grn.shape

In [ ]:
plt.imshow(np.corrcoef(filt_atacseq_obs))

In [4]:
print(grn.shape)
print(region_to_gene.shape)
print(region_to_tf.shape)
print(atacseq_obs.shape)


(110, 6)
(330, 110)
(330, 6)
(330, 5000)


In [ ]:
atac_perturbed = atac_base[:,-600:]
atac_unpert = atac_base[:,:-600]
atac_obs_perturbed = atacseq_obs[:,-600:]
atac_obs_unpert = atacseq_obs[:,:-600]
filt_atac_perturbed = filt_atacseq_obs[:,-600:]
filt_atac_unpert = filt_atacseq_obs[:,:-600]


In [ ]:
def above_threshold(matrix:np.array, percentile : int = 50, threshold : int = None):
    if threshold is None:
        threshold = np.percentile(matrix, percentile)
    return (matrix >= np.full(shape=matrix.shape,fill_value=threshold)).astype(int)

def normalize_matrix(df: np.array):
    if type(df) == np.ndarray:
        print("np.array")
        return df/sum(df)
    else:
        return df/df.sum().sum()

def distance(x1:np.array, x2: np.array):
    x1 = np.asarray(x1, dtype=np.float32)
    x2 = np.asarray(x2, dtype=np.float32)
    return np.sqrt(np.sum(np.square(np.subtract(x1, x2))))

def get_sparsity(mtx:np.ndarray):
    return np.count_nonzero(mtx)/np.prod(mtx.shape)

def add_noise(mtx:np.ndarray, mean=1, std=0.1):
    return mtx * np.random.normal(loc=mean, scale=std, size=mtx.shape)

def add_saltpepper(mtx:np.ndarray, p, min=None, max=None):
    if max is None:
        max = np.max(mtx)    
    if min is None:
        min = np.min(mtx)
    indexes = np.random.rand(*mtx.shape)<p
    mtx[indexes] = np.random.binomial(1, p=0.5, size=np.sum(indexes))
    return mtx


In [ ]:
def get_random_samples(n_samples, model: np.ndarray):
    sparsity = get_sparsity(model)
    size = model.shape
    # make binary atac
    binary = np.random.rand(n_samples, *size) < sparsity
    # fill binary with values
    uniques, counts = np.unique(model[model>0], return_counts=True)
    p = counts/counts.sum()

    values = np.random.choice(uniques, p=p, size=binary.sum())
    random_samples = np.zeros(shape=(n_samples, *size))
    random_samples[binary] = values
    return random_samples
#random_samples_atac = get_random_samples(n_samples=10000, model=atac_base)
#random_samples_grn = get_random_samples(n_samples=10000, model=grn)

In [ ]:
def get_mask(atac, region_to_gene, region_to_tf, threshold = False, threshold_kwargs = {}, correlation = False, correlation_kwargs = {}, pseudocounts = False, comment=None):
    """
    Args:
    atac (np.array): samples x regions
    params (dict): function parameters
    """
    if pseudocounts:
        atac += np.min(atac)*0.0001
    if correlation:
        corr_atac = np.abs(np.corrcoef(atac))
        if type(correlation_kwargs.get("mask", False)) is np.ndarray:
            corr_atac *= correlation_kwargs["mask"]
        if correlation_kwargs.get("normalize", False):
            corr_atac = normalize_matrix(corr_atac)
        if correlation_kwargs.get("threshold", False):
            corr_atac = above_threshold(corr_atac, **correlation_kwargs["threshold_kwargs"])
    else:
        corr_atac = atac @ atac.T
    mask = region_to_gene.T @ corr_atac @ region_to_tf
    if threshold:
        mask = above_threshold(mask, **threshold_kwargs)
    return mask
def get_mask_no_data(region_to_gene, region_to_tf, threshold = False, threshold_kwargs = {}, correlation = False, correlation_kwargs = {}, pseudocounts = False, comment=None):
    
    mask = region_to_gene.T @ region_to_tf
    if threshold:
        mask = above_threshold(mask, **threshold_kwargs)
    return mask.astype(np.float32)

In [ ]:
def evaluate_model(grn: np.array, atac: np.array, get_mask, funct_params: dict, runtime:float = None, n_samples: int=1000, complicated = True, random_samples:np.ndarray = np.array([])):
    """
    Evaluate the get_mask function based on the distance of its output to the grn.
    
    Args:
    runtime(float): Time in minutes, this function should be running for.
    """

    mask = get_mask(atac, **funct_params)
    mask_distance = distance(grn, mask)

    # distance distribution when atac is random
    sparsity = get_sparsity(grn)
    print(f"Sparsity: {sparsity}")
    if len(random_samples)>0:
        assert random_samples[0].shape == atac_base.shape, "Shape of random_samples not compatilble with atac matrix!"
        n_samples = len(random_samples)

    elif complicated:
        random_samples = get_random_samples(n_samples=n_samples, model=atac)
    else:
        random_samples = np.random.binomial(1, sparsity, size=(n_samples,)+atac.shape)

    time_per_getmask = timeit.timeit("get_mask(atac, **funct_params)", number=1, globals=locals())
    if runtime:
        runtime/=60
        n_samples = runtime//time_per_getmask

    print(f"Running model {n_samples} times to determine the random distribution.\nThis will take {time_per_getmask*n_samples /60} minutes.")
    rand_distances = np.empty(n_samples)
    for n, rand in enumerate(random_samples):
        mask = get_mask(rand, **funct_params)
        rand_distances[n] = distance(grn, mask)

    p_value = stats.percentileofscore(rand_distances, mask_distance)

    print("Distance to grn: ", mask_distance)
    print("p-value:", stats.percentileofscore(rand_distances, mask_distance))
    return mask_distance, p_value, rand_distances

#evaluate_model(grn=grn, atac=atac_base, get_mask=get_mask, funct_params=parameter_sets["params4"], n_samples=1)

In [ ]:
parameter_sets={
    "params1" : {
        "correlation":False,
        "threshold" : False,
        "comment" : "without correlation matrix"
    },
    "params2" : {
        "correlation":True,
        "threshold" : False,
        "comment":"with correlation matrix"
    },
    "params3" : {
        "correlation":True,
        "correlation_kwargs": {
            "threshold" : True,
            "threshold_kwargs": {
                "threshold": 0.999,
            },
        },
        "threshold" : False,
        "comment" : "no_data"
    },
    "params4" : {
        "correlation":False,
        "threshold" : True,
        "threshold_kwargs": {
            "percentile": 50,
            "threshold" : None,
        },
        "comment":"no correlation, threshold"
    },
    "params5" : {
        "pseudocounts":True,
        "correlation":False,
        "threshold" : True,
        "threshold_kwargs": {
            "percentile": 50,
            "threshold" : None,
        },
        "comment":"no correlation,\nthreshold,\npseudocounts"
    },
    "params6" : {
        "correlation":True,
        "correlation_kwargs": {
            "threshold" : True,
            "threshold_kwargs": {
                "percentile": 10,
                "threshold" : 0.4,
            },
        },
        "threshold" : True,
        "threshold_kwargs": {
            "percentile": 10,
            "threshold" : 1,
        },
        "comment":"correlation matrix thresholded,\nthreshold"
    },
    "params7" : {
        "correlation":True,
        "correlation_kwargs": {
            "threshold" : True,
            "threshold_kwargs": {
                "percentile": 10,
                "threshold" : 0.4,
            },
        },
        "threshold" : False,
        "comment":"correlation matrix thresholded"
    },
    "params8" : {
        "pseudocounts":True,
        "correlation":True,
        "correlation_kwargs": {
            "threshold" : True,
            "threshold_kwargs": {
                "percentile": 10,
                "threshold" : 0.4,
            },
        },
        "threshold" : False,
        "comment":"correlation matrix thresholded,\npseudocounts"
    },
    "params9" : {
        "correlation":True,
        "correlation_kwargs": {
            "normalize" : True,
            "threshold" : True,
            "threshold_kwargs": {
                "percentile": 10,
                "threshold" : 0.7,
            },
        },
        "threshold" : False,
        "comment":"correlation matrix thresholded normalized"
    },
    #"params10" : {
    #    "correlation":True,
    #    "correlation_kwargs": {
    #        "mask" : np.ones((50, 50)) - np.diag(np.ones(50)), # depends on atac data shape
    #        "normalize":True,
    #        "threshold" : False,
    #        "threshold_kwargs": {
    #            "percentile": 10,
    #            "threshold" : 0.4,
    #        },
    #    },
    #    "threshold" : False,
    #    "comment":"correlation matrix masked normalized"
    #}
}

corr = np.abs(np.corrcoef(atac_base))
params_corr_threshold = {n:{
        "correlation":True,
        "correlation_kwargs": {
            "threshold" : True,
            "threshold_kwargs": {
                "percentile": 10,
                "threshold" : n,
            },
        },
        "threshold" : False,
    } for n in np.arange(0, 1.1, 0.1)}

In [ ]:
masks ={key:get_mask(atac_base,
                     region_to_gene,
                     region_to_tf,
                     **params,
                     ) for key, params in parameter_sets.items()}

#plt.imshow(np.corrcoef(atacseq_obs, dtype=float)* np.diag(np.ones(len(atacseq_obs), dtype=float)))
#plt.xticks(np.arange(0, 2, 0.1))
print([distance(grn, mask) for mask in masks.values()])
fig, ax = plt.subplots(nrows=1, ncols=len(masks)+1)
ax = ax.flatten()
ax[0].imshow(grn)

ax[0].set_title("GRN")
ax[0].set_xlabel("TFs")
ax[0].set_ylabel("Genes")
for n, (key, mask) in enumerate(masks.items()):
    ax[n+1].imshow(mask)
    ax[n+1].set_xlabel("TFs")
    ax[n+1].set_ylabel("Genes")
    ax[n+1].set_title(parameter_sets[key]["comment"])

fig.suptitle("Different mask creation parameters")
fig.tight_layout()
#fig.subplots_adjust(top=0.88)
#fig.savefig(out_path/"figures"/"parameters_.pdf", format="pdf")

In [ ]:
# observe different noise parameters
np.random.seed(SEED)

repeats =3
distances = np.empty((10, 3, repeats))
for p in np.arange(repeats):
    for n in np.arange(0, 1, 0.1):
        region_to_gene_noised = add_noise(region_to_gene, 1, n)
        region_to_tf_noised = add_noise(region_to_tf, 1, n)
        print(n)
        masks ={key:get_mask(atac_obs_unpert,
                             region_to_gene=add_saltpepper(region_to_gene, p=n),
                             region_to_tf=add_saltpepper(region_to_tf, p=n),
                             **params,
                             ) for key, params in parameter_sets.items()}
        distances[int(n*10), :, p] = [distance(grn, mask) for mask in masks.values()]

std = distances.std(axis=2)
distances = distances.mean(axis=2)

#fig, ax = plt.subplots(nrows=1, ncols=3)
#ax = ax.flatten()
colors = ["green", "red", "blue"]
for n, c in enumerate(distances.T):
    plt.errorbar(np.arange(0, 1, 0.1), c,
             yerr=std.T[n],
             label=[x for x in parameter_sets.values()][n]["comment"], color=colors[n])
    plt.hlines(y=c.mean(), xmin=0, xmax=1, color=colors[n])
plt.legend()
#fig.align_ylabels()
plt.show()

In [ ]:
for key, params in parameter_sets.items():
    filename = key+".pickle"
    with open(out_path/"parameters"/filename, "wb") as handle:
        pickle.dump(params, file=handle)

In [ ]:

for key,params in parameter_sets.items():
    #if key not in ["params3","params6"]:
    #    continue
    mask = get_mask(atac_base, **params)
    fig, ax = plt.subplots(nrows=1, ncols=2)
    threshold = 1
    percentile = 50
    ax[0].imshow(grn)

    ax[1].imshow(mask)
    ax[1].set_title(params)
    fig.tight_layout()
    
    fig.show()
    fig, ax = plt.subplots(nrows=1, ncols=2)
    ax[0].hist(grn)
    ax[1].hist(mask)
    #plt.xticks(np.arange(-0.006, 0.025, 0.001))
    fig.tight_layout()
    fig.show()


In [ ]:
mask > 1

In [ ]:
grn>1

In [ ]:
plt.xlabel("Region")
plt.ylabel("Cells")
plt.imshow(atac_base)

In [ ]:
sc.pp.filter_cells(atac_base, min_genes=0)
sc.pp.filter_genes(atac_base, min_counts=0)

In [ ]:
atac_base.var_names

In [ ]:
sc.pl.violin(atac_base.T,
             keys="n_counts")

In [ ]:
sc.pl.violin(atac_base,
             keys="n_genes")